In [1]:
!pip install requests pandas ta schedule python-telegram-bot --quiet


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.0/731.0 kB 10.6 MB/s eta 0:00:00


In [2]:
import requests, pandas, ta, schedule
print("✅ All libraries working!")


✅ All libraries working!


In [3]:
import requests
import pandas as pd
from datetime import datetime

def get_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 10,   # top 10 coins
        "page": 1
    }
    data = requests.get(url, params=params).json()
    df = pd.DataFrame(data)[["id", "symbol", "current_price", "market_cap", "total_volume", "price_change_percentage_24h"]]
    df["time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return df

# Run it once
crypto_df = get_crypto_data()
crypto_df


,id,symbol,current_price,market_cap,total_volume,price_change_percentage_24h,time
0,bitcoin,btc,110473.000000,2201316894364,28034100895,0.51791,2025-11-02 06:46:19
1,ethereum,eth,3900.450000,470132674105,12875438537,1.26953,2025-11-02 06:46:19
2,tether,usdt,0.999798,183420837030,50437960841,0.01454,2025-11-02 06:46:19
3,ripple,xrp,2.520000,151411112992,1748272795,0.68829,2025-11-02 06:46:19
4,binancecoin,bnb,1090.420000,150017679818,962421604,0.56532,2025-11-02 06:46:19
5,solana,sol,186.980000,103221735749,2775001695,0.81339,2025-11-02 06:46:19
6,usd-coin,usdc,0.999799,75849398407,3857521577,0.00914,2025-11-02 06:46:19
7,staked-ether,steth,3894.540000,33425926284,11807267,1.13363,2025-11-02 06:46:19
8,dogecoin,doge,0.187307,28375786027,770912809,0.89448,2025-11-02 06:46:19
9,tron,trx,0.296110,28033421877,364381760,0.29021,2025-11-02 06:46:19


In [5]:
!pip install ta --quiet


In [6]:
import requests
import pandas as pd
import ta
from datetime import datetime

# ====== 1️⃣ Fetch Live Crypto Data ======
def get_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 10,
        "page": 1
    }
    data = requests.get(url, params=params).json()
    df = pd.DataFrame(data)[["id", "symbol", "current_price", "total_volume", "market_cap", "price_change_percentage_24h"]]
    df["time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return df


# ====== 2️⃣ Add Indicators ======
def add_indicators(df):
    close = df['current_price']

    # RSI
    df['rsi'] = ta.momentum.RSIIndicator(close, window=14).rsi()

    # Stochastic RSI
    df['stoch_rsi'] = ta.momentum.StochRSIIndicator(close, window=14).stochrsi()

    # EMA 20 and 50
    df['ema_20'] = ta.trend.EMAIndicator(close, window=20).ema_indicator()
    df['ema_50'] = ta.trend.EMAIndicator(close, window=50).ema_indicator()

    # MACD
    macd = ta.trend.MACD(close)
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()

    # Volume Spike Filter
    df['vol_avg'] = df['total_volume'].rolling(window=5).mean()
    df['volume_spike'] = df['total_volume'] > 1.5 * df['vol_avg']

    return df


# ====== 3️⃣ Generate Multi-Indicator Signal ======
def generate_signals(df):
    signals = []
    for _, row in df.iterrows():
        if (
            row['rsi'] < 35 and
            row['stoch_rsi'] < 0.2 and
            row['current_price'] > row['ema_20'] > row['ema_50'] and
            row['macd'] > row['macd_signal'] and
            row['volume_spike']
        ):
            signals.append('🔥 HIGH-CONFIDENCE BUY')
        elif (
            row['rsi'] > 70 and
            row['stoch_rsi'] > 0.8 and
            row['current_price'] < row['ema_20'] < row['ema_50'] and
            row['macd'] < row['macd_signal']
        ):
            signals.append('🚨 STRONG SELL')
        else:
            signals.append('HOLD')
    df['signal'] = signals
    return df


# ====== 4️⃣ Run the system ======
crypto_df = get_crypto_data()
crypto_df = add_indicators(crypto_df)
crypto_df = generate_signals(crypto_df)

# Show key columns
crypto_df[['id', 'symbol', 'current_price', 'rsi', 'stoch_rsi', 'ema_20', 'ema_50', 'macd', 'macd_signal', 'volume_spike', 'signal']]


,id,symbol,current_price,rsi,stoch_rsi,ema_20,ema_50,macd,macd_signal,volume_spike,signal
0,bitcoin,btc,110661.000000,NaN,NaN,NaN,NaN,NaN,NaN,False,HOLD
1,ethereum,eth,3902.030000,NaN,NaN,NaN,NaN,NaN,NaN,False,HOLD
2,tether,usdt,0.999825,NaN,NaN,NaN,NaN,NaN,NaN,False,HOLD
3,ripple,xrp,2.520000,NaN,NaN,NaN,NaN,NaN,NaN,False,HOLD
4,binancecoin,bnb,1091.750000,NaN,NaN,NaN,NaN,NaN,NaN,False,HOLD
5,solana,sol,187.390000,NaN,NaN,NaN,NaN,NaN,NaN,False,HOLD
6,usd-coin,usdc,0.999707,NaN,NaN,NaN,NaN,NaN,NaN,False,HOLD
7,staked-ether,steth,3902.260000,NaN,NaN,NaN,NaN,NaN,NaN,False,HOLD
8,dogecoin,doge,0.187691,NaN,NaN,NaN,NaN,NaN,NaN,False,HOLD
9,tron,trx,0.296345,NaN,NaN,NaN,NaN,NaN,NaN,False,HOLD


In [7]:
import pandas as pd
import requests
import numpy as np

# Function to fetch live crypto data
def get_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {"vs_currency": "usd", "order": "market_cap_desc", "per_page": 10, "page": 1}
    data = requests.get(url, params=params).json()
    df = pd.DataFrame(data)[["id", "symbol", "current_price", "total_volume", "price_change_percentage_24h"]]
    return df

# Function to calculate RSI
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = delta.clip(lower=0)
    loss = -1 * delta.clip(upper=0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Main Analyzer
def analyze_crypto():
    df = get_crypto_data()

    # Simulate small price movements to compute RSI (since API gives current price only)
    df["price_history"] = df["current_price"].apply(lambda x: np.linspace(x*0.98, x*1.02, 30))
    df["RSI"] = df["price_history"].apply(lambda x: calculate_rsi(pd.Series(x)).iloc[-1])

    # Trend strength using simple EMA rule
    df["EMA_short"] = df["current_price"] * 0.99
    df["EMA_long"] = df["current_price"] * 1.01
    df["Trend"] = np.where(df["EMA_short"] > df["EMA_long"], "Bullish", "Bearish")

    # Volume strength
    avg_vol = df["total_volume"].mean()
    df["Volume_Strength"] = np.where(df["total_volume"] > avg_vol, "High", "Low")

    # Final signal
    conditions = []
    for _, row in df.iterrows():
        if row["RSI"] < 30 and row["Trend"] == "Bullish" and row["Volume_Strength"] == "High":
            conditions.append("STRONG BUY ✅")
        elif 30 <= row["RSI"] <= 70:
            conditions.append("NEUTRAL ⚠️")
        else:
            conditions.append("SELL 🔻")
    df["Signal"] = conditions

    return df[["id", "symbol", "current_price", "RSI", "Trend", "Volume_Strength", "Signal"]]

# Run analysis
analyze_crypto()


,id,symbol,current_price,RSI,Trend,Volume_Strength,Signal
0,bitcoin,btc,110640.000000,100.0,Bearish,High,SELL 🔻
1,ethereum,eth,3901.790000,100.0,Bearish,High,SELL 🔻
2,tether,usdt,0.999811,100.0,Bearish,High,SELL 🔻
3,ripple,xrp,2.520000,100.0,Bearish,Low,SELL 🔻
4,binancecoin,bnb,1091.730000,100.0,Bearish,Low,SELL 🔻
5,solana,sol,187.410000,100.0,Bearish,Low,SELL 🔻
6,usd-coin,usdc,0.999768,100.0,Bearish,Low,SELL 🔻
7,staked-ether,steth,3900.970000,100.0,Bearish,Low,SELL 🔻
8,dogecoin,doge,0.187689,100.0,Bearish,Low,SELL 🔻
9,tron,trx,0.296337,100.0,Bearish,Low,SELL 🔻


In [9]:
import requests

# Replace these with your details
BOT_TOKEN = 8376149890
CHAT_ID =1609197089

def send_telegram_alert(message):
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    payload = {"chat_id": CHAT_ID, "text": message}
    requests.post(url, data=payload)

def run_and_alert():
    df = analyze_crypto()
    strong_buys = df[df["Signal"].str.contains("STRONG BUY")]
    if not strong_buys.empty:
        msg = "🚀 STRONG BUY ALERT 🚀\n\n"
        for _, row in strong_buys.iterrows():
            msg += f"{row['id'].upper()} ({row['symbol'].upper()})\nPrice: ${row['current_price']}\nRSI: {row['RSI']:.2f}\nTrend: {row['Trend']}\n\n"
        send_telegram_alert(msg)
    else:
        print("No strong buy signals now.")

# Run it
run_and_alert()


No strong buy signals now.


In [12]:
import requests

BOT_TOKEN =8376149890   # keep your real one here
CHAT_ID =1609197089      # number you found from getUpdates

url = f"https://api.telegram.org/bot{8376149890}/sendMessage"
payload = {"chat_id": 1609197089, "text": "🔥 Test message from your crypto system!"}

response = requests.post(url, data=payload)
print(response.status_code, response.text)


404 {"ok":false,"error_code":404,"description":"Not Found"}


In [13]:
import requests

BOT_TOKEN = 8376149890   # keep your real one here
CHAT_ID = 1609197089      # number you found from getUpdates

url = f"https://api.telegram.org/bot8376149890:AAFiw5rok3-NbT5SdxHGWcmn3Q7aEOzKKYs/sendMessage"
payload = {"chat_id": 1609197089 , "text": "🔥 Test message from your crypto system!"}

response = requests.post(url, data=payload)
print(response.status_code, response.text)


200 {"ok":true,"result":{"message_id":9,"from":{"id":8376149890,"is_bot":true,"first_name":"ManuCryptoAlertBot","username":"ManuCryptoAlertBot"},"chat":{"id":1609197089,"first_name":"Manu","last_name":"Manoranjan","type":"private"},"date":1762068883,"text":"\ud83d\udd25 Test message from your crypto system!"}}


In [ ]:
import time

def run_auto(interval_minutes=30):
    print(f"🤖 Auto mode started. Checking every {interval_minutes} minutes...\n")
    while True:
        run_and_alert()
        print(f"⏱ Waiting {interval_minutes} minutes before next scan...\n")
        time.sleep(interval_minutes * 60)

# Start automation (you can change 30 to 10 or 60 as you like)
run_auto(30)


🤖 Auto mode started. Checking every 30 minutes...

No strong buy signals now.
⏱ Waiting 30 minutes before next scan...

